# Test evolving the $^{3}S_1$ channel with no coupling to $^{3}D_1$

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ March 7, 2023

This notebook tests the momentum distribution calculation in the $^{3}S_1$ channel with no coupling to $^{3}D_1$.

_Last update:_ March 8, 2023

In [1]:
# Python imports
import numpy as np
from scipy.integrate import ode
import time

In [25]:
# Imports from scripts
# from scripts.figures import set_rc_parameters
from scripts.potentials import Potential
from scripts.tools import replace_periods

In [3]:
# Run this cell to turn on customized matplotlib graphics
# set_rc_parameters()

## Set-up

In [4]:
kvnn = 6  # AV18
channel = '3S1'
kmax, kmid, ntot = 15.0, 3.0, 120
generator = 'Wegner'
lamb = 1.35

In [5]:
potential = Potential(kvnn, channel, kmax, kmid, ntot)

In [6]:
# Initial Hamiltonian with integration factors attached [MeV]
H_initial_weights = potential.load_hamiltonian()

## Set $^{3}S_1-^{3}D_1$ and $^{3}D_1-^{3}S_1$ off-diagonal blocks to zero

In [7]:
H_initial_weights[:ntot, ntot:] = np.zeros((ntot, ntot))
H_initial_weights[ntot:, :ntot] = np.zeros((ntot, ntot))

## SRG evolve the potential

Using `scipy.integrate.ode` and solving for $H(\lambda)$ w.r.t. $\lambda$.

In [8]:
def matrix_to_vector(M):
    """
    Takes the upper triangle of the matrix M (including the diagonal) and
    reshapes it into a vector v.
        
    Parameters
    ----------
    M : 2-D ndarray
        Input matrix of shape (N, N).
            
    Returns
    -------
    v : 1-D ndarray
        Output vector of shape (N*(N+1)/2,).
            
    """

    # Length of matrix
    N = len(M)
    # Length of vectorized matrix
    n = int(N * (N + 1) / 2)

    # Initialize vectorized matrix
    v = np.zeros(n)

    # Algorithm for reshaping M to the vector v
    i = 0
    j = N
    for k in range(N):
        v[i:j] = M[k][k:]
        i = j
        j += N - k - 1

    return v

In [9]:
def vector_to_matrix(v):
    """
    Takes the vector of an upper triangle matrix v and returns the full matrix
    M. Use only for symmetric matrices.
        
    Parameters
    ----------
    v : 1-D ndarray
        Input vector of shape (N*(N+1)/2,).
        
    Returns
    -------
    output : 2-D ndarray
        Output matrix of shape (N, N).
            
    """
    
    # Dimension of matrix is found by solving for the positive solution to
    # n = N(N+1)/2, where n is the length of the vector
    N = int((-1 + np.sqrt(1 + 8 * len(v))) / 2)

    # Initialize matrix
    M = np.zeros((N, N))

    # Build the upper half of matrix with the diagonal included

    # Algorithm for reshaping v to the matrix M
    i = 0
    j = N
    for k in range(N):
        M[k, k:] = v[i:j]
        i = j
        j += N - k - 1

    # Now reflect the upper half to lower half to build full matrix
    # M.T - np.diag(np.diag(M)) is the lower half of M excluding diagonal
    return M + (M.T - np.diag(np.diag(M)))

In [10]:
def commutator(A, B):
    """Commutator of square matrices A and B."""

    return A @ B - B @ A

In [11]:
def eta(H_matrix):
    """Wegner generator \eta = [H_D, H]."""

    # G = H_D (diagonal of the evolving Hamiltonian)
    G_matrix = np.diag(np.diag(H_matrix))

    # \eta = [G, H]
    return commutator(G_matrix, H_matrix)

In [12]:
def H_deriv(lamb, H_vector):
    """Right-hand side of the SRG flow equation."""

    # Matrix form of the evolving Hamiltonian
    H_matrix = vector_to_matrix(H_vector)

    # Get SRG generator \eta = [G, H]
    eta_matrix = eta(H_matrix)

    # RHS of the flow equation in matrix form
    dH_matrix = -4.0 / lamb ** 5 * commutator(eta_matrix, H_matrix)

    # Returns vector form of RHS of flow equation
    dH_vector = matrix_to_vector(dH_matrix)

    return dH_vector

In [13]:
def select_step_size(solver_lambda, lambda_final):
    """Select ODE solver step-size depending on the extent of evolution. We
    can take bigger steps at large values of \lambda.
    """

    if solver_lambda >= 6.0:
        dlamb = 1.0
    elif 2.5 <= solver_lambda < 6.0:
        dlamb = 0.5
    elif 1.5 <= solver_lambda < 2.5:
        dlamb = 0.1
    else:
        dlamb = 0.05

    return dlamb

In [14]:
def get_ode_solver(lambda_initial, H_initial, atol, rtol):
    """Sets up the ODE solver."""

    # Solving for H(s)
    solver = ode(H_deriv)

    # Initial Hamiltonian as a vector
    H_initial = matrix_to_vector(H_initial)

    # Set initial conditions
    solver.set_initial_value(H_initial, lambda_initial)

    # Following the example in Hergert:2016iju with modifications to nsteps and
    # error tolerances
    solver.set_integrator('vode', method='bdf', order=5, atol=atol, rtol=rtol,
                          nsteps=5000000)

    return solver

In [15]:
def srg_evolve_wrt_lambda(
        H_initial_MeV, lambda_array, lambda_initial=20.0, atol=1e-10, rtol=1e-10
):
    """SRG evolve the Hamiltonian with respect to \lambda."""
    
    # Convert Hamiltonian from MeV to units [fm^-2]
    H_initial = H_initial_MeV / 41.47

    # Set-up ODE solver
    solver = get_ode_solver(lambda_initial, H_initial, atol, rtol)
    
    # Start time
    t0 = time.time()
    
    # Evolve the Hamiltonian to each value of \lambda and store in dictionary
    d = {}
    for lamb in lambda_array:

        # Solve ODE up to lamb and store in dictionary
        while solver.successful() and round(solver.t, 2) > lamb:
            
            # Get ODE solver step-size in \lambda
            dlamb = select_step_size(solver.t, lamb)
            
            # Integrate to next step in lambda
            solution_vector = solver.integrate(solver.t - dlamb)

        # Store evolved Hamiltonian matrix [MeV] in dictionary
        d[lamb] = vector_to_matrix(solution_vector) * 41.47

    # End time
    t1 = time.time()

    # Print details
    mins = round((t1 - t0) / 60.0, 4)  # Minutes elapsed evolving H(\lambda)
    print(f"Done evolving to \lambda = {lamb} fm^-1 after {mins:.4f} minutes.")

    return d

In [16]:
lambda_array = np.array([6.0, 3.0, 2.0, 1.5, 1.35])
d = srg_evolve_wrt_lambda(H_initial_weights, lambda_array, lambda_initial=20.0)

Done evolving to \lambda = 1.35 fm^-1 after 1.9700 minutes.


In [17]:
# Get H(\lambda=1.35)
H_evolved_weights = d[1.35]

## Save evolved Hamiltonian for `test_momentum_distribution_script.py`

In [26]:
def save_H_evolved(d, kvnn, kmax, kmid, ntot, generator):

    for lamb in d:
            
        file_name = (
            f"H_evolved_kvnn_{kvnn}_3S1_3D1_no_coupling_{generator}_lamb_{lamb}"
            f"_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_ode_BDF_wrt_lambda"
        )
            
        np.savetxt("./test_srg/" + replace_periods(file_name) + ".txt", d[lamb])

In [27]:
# Copy this into test_momentum_distribution_script.py and use in place of 
# potential.load_hamiltonian or load_H_evolved
def load_H_3S1_no_coupling(
    kvnn, generator, lamb, kmax=15.0, kmid=3.0, ntot=120
):
    
    file_name = (
        f"H_evolved_kvnn_{kvnn}_3S1_3D1_no_coupling_{generator}_lamb_{lamb}"
        f"_kmax_{kmax}_kmid_{kmid}_ntot_{ntot}_ode_BDF_wrt_lambda"
    )
    
    H_evolved = np.loadtxt("./test_srg/" + replace_periods(file_name) + ".txt")
    
    return H_evolved

In [28]:
save_H_evolved(d, kvnn, kmax, kmid, ntot, generator)

In [ ]:
# Test the loading function
H_temp = load_H_3S1_no_coupling(kvnn, generator, lamb)